In [1]:
# パス指定 Sparkではdbfs:を使用してアクセス
source_path = 'dbfs:/databricks-datasets/samples/lending_club/parquet/'
delta_path = 'dbfs:/mnt/analytics_data/lending_data/delta/'

dbutils.fs.ls(delta_path)

Out[2]: [FileInfo(path='dbfs:/mnt/analytics_data/lending_data/delta/_delta_log/', name='_delta_log/', size=0),
 FileInfo(path='dbfs:/mnt/analytics_data/lending_data/delta/part-00000-7ce70dc4-6973-48b5-876f-ce1b77b397d7-c000.snappy.parquet', name='part-00000-7ce70dc4-6973-48b5-876f-ce1b77b397d7-c000.snappy.parquet', size=72231037),
 FileInfo(path='dbfs:/mnt/analytics_data/lending_data/delta/part-00001-25b906af-8d26-4ac7-826d-eaa5293e1e5d-c000.snappy.parquet', name='part-00001-25b906af-8d26-4ac7-826d-eaa5293e1e5d-c000.snappy.parquet', size=57787171),
 FileInfo(path='dbfs:/mnt/analytics_data/lending_data/delta/part-00002-e9794650-9408-4245-9fe1-3504945e1bb1-c000.snappy.parquet', name='part-00002-e9794650-9408-4245-9fe1-3504945e1bb1-c000.snappy.parquet', size=43383132),
 FileInfo(path='dbfs:/mnt/analytics_data/lending_data/delta/part-00003-99944a3f-7852-4bb9-a65a-492b32607d8d-c000.snappy.parquet', name='part-00003-99944a3f-7852-4bb9-a65a-492b32607d8d-c000.snappy.parquet', size=21870935)]

In [2]:
# 既存のデータを削除
# Deltaテーブルのパスを削除。
dbutils.fs.rm(delta_path, True)

# ソースディレクトリにあるParquetファイルをデータフレームに読み込む
df = spark.read.parquet(source_path)

In [3]:
len(df.columns)

Out[7]: 137

In [4]:
#ここではrandomSplit()を使って、ワークショップでは10%のサンプルを使用して実行
(data, data_rest) = df.randomSplit([0.10, 0.90], seed=123)

# 読み込まれたデータを参照
display(data)

# レコード件数確認
print("全レコード件数:" , df.count())
print("ワークショップで使用するレコード件数:" , data.count())

# 物理パスの位置
print('delta_path:' + delta_path)

In [5]:
username = "takaakiyayoi"

In [6]:
# データベースを作成。
spark.sql(f"create database {username}_db")
# データベースを使用。
spark.sql(f"use {username}_db")

Out[5]: DataFrame[]

In [7]:
%sql
drop table orijinal_data

In [8]:
table_name = "original_data"

In [9]:
%python
# データフレームのデータをdeltaとして登録
df.write.format('delta').mode("overwrite").option("path", delta_path).saveAsTable(table_name)

In [10]:
df.write.format('delta').mode("append").option("path", delta_path).saveAsTable(table_name)

In [11]:
display(spark.sql(f"DROP TABLE IF EXISTS {table_name}"))
display(spark.sql(f"CREATE TABLE {table_name} USING DELTA LOCATION '{delta_path}'"))

In [12]:
%sql
SELECT COUNT(*) FROM original_data

count(1)
1481560


In [13]:
df = sql(
  '''
  Select 
  *
  From orijinal_data
  where 1=1
  order by random()
  limit 1000000
  '''
)
display(df)

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_il_6m,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,issue_d
null,null,4800.0,4800,4800.0,36 months,13.99%,164.03,C,C4,Teacher,10+ years,MORTGAGE,51000.0,Verified,Current,n,null,null,debt_consolidation,Debt consolidation,797xx,TX,21.69,1.0,Sep-1990,0,22,null,14,0,3576,27.5%,45.0,w,1968.03,1968.03,3765.23,3765.23,2831.97,933.26,0.0,0.0,0.0,Aug-2017,164.03,Sep-2017,Aug-2017,0,22,1,INDIVIDUAL,null,null,null,0,0,83811,null,null,null,null,null,null,null,null,null,null,null,13000,null,null,null,19,5987,3405,24.3,0.0,0,219.0,300,2,2,1,4,24,13,24,2,2,5,2,4,28,10,16,5,14,0,0,1,11,83.7,50.0,0,0,102662,14832,4500,11620,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Sep-2015
null,null,7000.0,7000,7000.0,36 months,12.62%,234.58,C,C1,Program Director,10+ years,RENT,50000.0,Source Verified,Current,n,null,null,other,Other,021xx,MA,22.64,0.0,Jan-2008,2,30,null,12,0,4721,43.3%,23.0,f,6676.38,6676.38,464.25,464.25,323.62,140.63,0.0,0.0,0.0,Aug-2017,234.58,Sep-2017,Aug-2017,0,30,1,INDIVIDUAL,null,null,null,0,0,34999,0,9,0,0,59,30278,66,0,3,1749,62,10900,1,0,2,3,2917,51,97.2,0.0,0,104.0,113,14,14,0,22,54,0,54,10,1,3,1,2,18,3,5,3,12,0,0,0,0,54.5,100.0,0,0,56954,34999,1800,46054,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Jun-2017
null,null,12250.0,12250,12250.0,36 months,13.98%,418.56,C,C3,null,n/a,RENT,50000.0,Verified,Fully Paid,n,null,null,debt_consolidation,Debt consolidation,600xx,IL,25.98,0.0,Apr-1975,1,null,68,13,1,8295,54.6%,45.0,w,0.00,0.0,15055.8603173798,15055.86,12250.0,2805.86,0.0,0.0,0.0,Mar-2017,1243.38,null,May-2017,0,null,1,INDIVIDUAL,null,null,null,0,0,33641,null,null,null,null,null,null,null,null,null,null,null,15200,null,null,null,7,3364,969,3.1,0.0,0,141.0,469,6,6,2,10,null,0,null,0,1,7,1,12,12,10,31,7,13,0,0,0,4,100.0,0.0,1,

In [14]:
df.count()

Out[8]: 1000000

In [15]:
df = sql(
  '''
  Select 
  *
  From orijinal_data
  where 1=1
  order by random()
  limit 1000000
  '''
)
display(df)